# Dynamic Equilibrium Analysis

Explores stability and time-to-equilibrium questions for Cardano staking game theory.

## Research Questions

1. **Perturbation recovery** — how fast does the system recover from shocks?
2. **Dynamic margin competition** — what if operators adapt margins over time?
3. **Network growth** — what happens as total stake and pool count grows?
4. **Nakamoto coefficient over time** — centralization dynamics

## Mapping to Lean 4 Proofs

Maps to multiple \ statements in the formal verification:
- \ (convergence → existence)
- \ (Nakamoto coefficient)
- \ (approximate dynamics)

See \ for proofs.

## 1. Setup

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from cardano_staking_sim import (PoolParams, StakePool, pool_rewards,
                                  operator_rewards, delegator_reward_per_ada)
import os

## 2. Perturbation Recovery Simulation

Test how quickly the system recovers from a shock where 30% of delegators are randomly reassigned.